# 第4章：Spark SQL 使用指南

本章節將深入學習 Spark SQL 的使用，包括 SQL 語法、臨時視圖、複雜查詢等。

## 學習目標
- 掌握 Spark SQL 的基本語法
- 學習創建和使用臨時視圖
- 理解 DataFrame 和 SQL 的對應關係
- 學習複雜查詢和優化技巧

## 1. 環境設置

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, avg, sum as spark_sum, count, max as spark_max, min as spark_min
from pyspark.sql.types import *
from datetime import datetime, date
import pandas as pd

In [ ]:
# 創建 SparkSession
spark = SparkSession.builder \
    .appName("SparkSQL學習") \
    .master("local[*]") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .getOrCreate()

print(f"Spark版本: {spark.version}")
print(f"Spark UI: http://localhost:4040")

## 2. 準備測試數據

In [ ]:
# 創建員工數據
employees_data = [
    (1, "Alice", 25, "Engineering", 75000, "2020-01-15", "alice@company.com"),
    (2, "Bob", 30, "Sales", 65000, "2019-03-20", "bob@company.com"),
    (3, "Charlie", 35, "Engineering", 85000, "2018-07-10", "charlie@company.com"),
    (4, "Diana", 28, "Marketing", 60000, "2021-02-05", "diana@company.com"),
    (5, "Eve", 32, "Engineering", 80000, "2019-11-12", "eve@company.com"),
    (6, "Frank", 29, "Sales", 70000, "2020-08-30", "frank@company.com"),
    (7, "Grace", 26, "Marketing", 55000, "2021-05-18", "grace@company.com"),
    (8, "Henry", 31, "Engineering", 90000, "2017-04-22", "henry@company.com"),
    (9, "Ivy", 27, "Sales", 68000, "2020-10-15", "ivy@company.com"),
    (10, "Jack", 33, "Marketing", 62000, "2019-12-01", "jack@company.com")
]

employees_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("department", StringType(), True),
    StructField("salary", IntegerType(), True),
    StructField("hire_date", StringType(), True),
    StructField("email", StringType(), True)
])

employees_df = spark.createDataFrame(employees_data, employees_schema)
employees_df.show()

In [ ]:
# 創建部門數據
departments_data = [
    ("Engineering", "Tech", "Building A", 101),
    ("Sales", "Business", "Building B", 102),
    ("Marketing", "Business", "Building C", 103),
    ("HR", "Support", "Building D", 104),
    ("Finance", "Support", "Building E", 105)
]

departments_df = spark.createDataFrame(departments_data, 
                                     ["name", "division", "location", "manager_id"])
departments_df.show()

# 創建項目數據
projects_data = [
    (1, "Project Alpha", "Engineering", "2021-01-01", "2021-12-31", "Active"),
    (2, "Project Beta", "Sales", "2020-06-01", "2021-06-30", "Completed"),
    (3, "Project Gamma", "Marketing", "2021-03-01", "2021-09-30", "Active"),
    (4, "Project Delta", "Engineering", "2020-01-01", "2020-12-31", "Completed"),
    (5, "Project Epsilon", "Sales", "2021-07-01", "2022-06-30", "Active")
]

projects_df = spark.createDataFrame(projects_data, 
                                  ["id", "name", "department", "start_date", "end_date", "status"])
projects_df.show()

## 3. 創建臨時視圖

In [ ]:
# 創建臨時視圖
employees_df.createOrReplaceTempView("employees")
departments_df.createOrReplaceTempView("departments")
projects_df.createOrReplaceTempView("projects")

# 查看所有臨時視圖
spark.sql("SHOW TABLES").show()

print("臨時視圖創建完成！")

## 4. 基本 SQL 查詢

In [ ]:
# 基本 SELECT 查詢
print("=== 基本 SELECT 查詢 ===")
result = spark.sql("""
    SELECT name, age, department, salary
    FROM employees
    ORDER BY salary DESC
    LIMIT 5
""")
result.show()

# 使用 WHERE 條件
print("\n=== WHERE 條件查詢 ===")
result = spark.sql("""
    SELECT name, age, department, salary
    FROM employees
    WHERE department = 'Engineering' AND salary > 75000
""")
result.show()

In [ ]:
# 聚合查詢
print("=== 聚合查詢 ===")
result = spark.sql("""
    SELECT 
        department,
        COUNT(*) as employee_count,
        AVG(salary) as avg_salary,
        MAX(salary) as max_salary,
        MIN(salary) as min_salary,
        SUM(salary) as total_salary
    FROM employees
    GROUP BY department
    ORDER BY avg_salary DESC
""")
result.show()

# 使用 HAVING 子句
print("\n=== HAVING 子句 ===")
result = spark.sql("""
    SELECT 
        department,
        COUNT(*) as employee_count,
        AVG(salary) as avg_salary
    FROM employees
    GROUP BY department
    HAVING COUNT(*) > 2
    ORDER BY avg_salary DESC
""")
result.show()

## 5. 複雜查詢和子查詢

In [ ]:
# 子查詢
print("=== 子查詢 ===")
result = spark.sql("""
    SELECT name, department, salary
    FROM employees
    WHERE salary > (
        SELECT AVG(salary) 
        FROM employees
    )
    ORDER BY salary DESC
""")
result.show()

# 相關子查詢
print("\n=== 相關子查詢 ===")
result = spark.sql("""
    SELECT e1.name, e1.department, e1.salary
    FROM employees e1
    WHERE e1.salary > (
        SELECT AVG(e2.salary)
        FROM employees e2
        WHERE e2.department = e1.department
    )
    ORDER BY e1.department, e1.salary DESC
""")
result.show()

In [ ]:
# EXISTS 子查詢
print("=== EXISTS 子查詢 ===")
result = spark.sql("""
    SELECT d.name, d.division, d.location
    FROM departments d
    WHERE EXISTS (
        SELECT 1
        FROM employees e
        WHERE e.department = d.name
    )
""")
result.show()

# IN 子查詢
print("\n=== IN 子查詢 ===")
result = spark.sql("""
    SELECT name, department, salary
    FROM employees
    WHERE department IN (
        SELECT name
        FROM departments
        WHERE division = 'Tech'
    )
""")
result.show()

## 6. 連接查詢

In [ ]:
# INNER JOIN
print("=== INNER JOIN ===")
result = spark.sql("""
    SELECT 
        e.name,
        e.department,
        e.salary,
        d.division,
        d.location
    FROM employees e
    INNER JOIN departments d ON e.department = d.name
    ORDER BY e.salary DESC
""")
result.show()

# LEFT JOIN
print("\n=== LEFT JOIN ===")
result = spark.sql("""
    SELECT 
        e.name,
        e.department,
        e.salary,
        d.division,
        d.location
    FROM employees e
    LEFT JOIN departments d ON e.department = d.name
    ORDER BY e.name
""")
result.show()

In [ ]:
# 三表連接
print("=== 三表連接 ===")
result = spark.sql("""
    SELECT 
        e.name as employee_name,
        e.department,
        e.salary,
        d.division,
        d.location,
        p.name as project_name,
        p.status as project_status
    FROM employees e
    INNER JOIN departments d ON e.department = d.name
    INNER JOIN projects p ON e.department = p.department
    WHERE p.status = 'Active'
    ORDER BY e.department, e.name
""")
result.show()

## 7. 窗口函數

In [ ]:
# ROW_NUMBER, RANK, DENSE_RANK
print("=== 窗口函數 - 排名 ===")
result = spark.sql("""
    SELECT 
        name,
        department,
        salary,
        ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as row_num,
        RANK() OVER (PARTITION BY department ORDER BY salary DESC) as rank,
        DENSE_RANK() OVER (PARTITION BY department ORDER BY salary DESC) as dense_rank
    FROM employees
    ORDER BY department, salary DESC
""")
result.show()

# 找出每個部門薪水最高的員工
print("\n=== 每個部門薪水最高的員工 ===")
result = spark.sql("""
    SELECT name, department, salary
    FROM (
        SELECT 
            name,
            department,
            salary,
            ROW_NUMBER() OVER (PARTITION BY department ORDER BY salary DESC) as rn
        FROM employees
    ) ranked
    WHERE rn = 1
    ORDER BY department
""")
result.show()

In [ ]:
# LAG 和 LEAD 函數
print("=== LAG 和 LEAD 函數 ===")
result = spark.sql("""
    SELECT 
        name,
        department,
        salary,
        LAG(salary, 1) OVER (PARTITION BY department ORDER BY salary) as prev_salary,
        LEAD(salary, 1) OVER (PARTITION BY department ORDER BY salary) as next_salary,
        salary - LAG(salary, 1) OVER (PARTITION BY department ORDER BY salary) as salary_diff
    FROM employees
    ORDER BY department, salary
""")
result.show()

# 累計值
print("\n=== 累計值 ===")
result = spark.sql("""
    SELECT 
        name,
        department,
        salary,
        SUM(salary) OVER (PARTITION BY department ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as cumulative_salary,
        AVG(salary) OVER (PARTITION BY department ORDER BY salary ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as running_avg
    FROM employees
    ORDER BY department, salary
""")
result.show()

## 8. 公用表表達式 (CTE)

In [ ]:
# 單個 CTE
print("=== 單個 CTE ===")
result = spark.sql("""
    WITH high_earners AS (
        SELECT name, department, salary
        FROM employees
        WHERE salary > 70000
    )
    SELECT 
        department,
        COUNT(*) as high_earner_count,
        AVG(salary) as avg_high_salary
    FROM high_earners
    GROUP BY department
    ORDER BY avg_high_salary DESC
""")
result.show()

# 多個 CTE
print("\n=== 多個 CTE ===")
result = spark.sql("""
    WITH 
    dept_stats AS (
        SELECT 
            department,
            COUNT(*) as emp_count,
            AVG(salary) as avg_salary
        FROM employees
        GROUP BY department
    ),
    top_depts AS (
        SELECT department, avg_salary
        FROM dept_stats
        WHERE emp_count > 2
    )
    SELECT 
        e.name,
        e.department,
        e.salary,
        t.avg_salary as dept_avg_salary,
        e.salary - t.avg_salary as salary_diff
    FROM employees e
    INNER JOIN top_depts t ON e.department = t.department
    ORDER BY e.department, e.salary DESC
""")
result.show()

## 9. 條件表達式

In [ ]:
# CASE WHEN 表達式
print("=== CASE WHEN 表達式 ===")
result = spark.sql("""
    SELECT 
        name,
        department,
        age,
        salary,
        CASE 
            WHEN salary >= 85000 THEN 'Senior'
            WHEN salary >= 70000 THEN 'Mid'
            ELSE 'Junior'
        END as salary_level,
        CASE 
            WHEN age < 28 THEN 'Young'
            WHEN age < 32 THEN 'Middle'
            ELSE 'Senior'
        END as age_group
    FROM employees
    ORDER BY department, salary DESC
""")
result.show()

# 複雜的條件邏輯
print("\n=== 複雜條件邏輯 ===")
result = spark.sql("""
    SELECT 
        name,
        department,
        salary,
        CASE 
            WHEN department = 'Engineering' AND salary > 80000 THEN 'Tech Lead'
            WHEN department = 'Sales' AND salary > 65000 THEN 'Sales Manager'
            WHEN department = 'Marketing' AND salary > 55000 THEN 'Marketing Manager'
            ELSE 'Individual Contributor'
        END as role_level
    FROM employees
    ORDER BY department, salary DESC
""")
result.show()

## 10. 字符串函數

In [ ]:
# 常用字符串函數
print("=== 字符串函數 ===")
result = spark.sql("""
    SELECT 
        name,
        email,
        UPPER(name) as name_upper,
        LOWER(name) as name_lower,
        LENGTH(name) as name_length,
        SUBSTR(name, 1, 3) as name_prefix,
        CONCAT(name, ' - ', department) as name_dept,
        SPLIT(email, '@')[0] as username,
        SPLIT(email, '@')[1] as domain
    FROM employees
    ORDER BY name
""")
result.show()

# 字符串匹配
print("\n=== 字符串匹配 ===")
result = spark.sql("""
    SELECT 
        name,
        email,
        department
    FROM employees
    WHERE name LIKE '%a%'  -- 包含字母 'a'
       OR email LIKE '%gmail%'  -- 包含 'gmail'
       OR department RLIKE '^[ES].*'  -- 以 E 或 S 開頭
    ORDER BY name
""")
result.show()

## 11. 日期和時間函數

In [ ]:
# 日期函數
print("=== 日期函數 ===")
result = spark.sql("""
    SELECT 
        name,
        hire_date,
        TO_DATE(hire_date, 'yyyy-MM-dd') as hire_date_parsed,
        YEAR(TO_DATE(hire_date, 'yyyy-MM-dd')) as hire_year,
        MONTH(TO_DATE(hire_date, 'yyyy-MM-dd')) as hire_month,
        DAYOFWEEK(TO_DATE(hire_date, 'yyyy-MM-dd')) as hire_day_of_week,
        DATEDIFF(CURRENT_DATE(), TO_DATE(hire_date, 'yyyy-MM-dd')) as days_since_hire,
        DATE_ADD(TO_DATE(hire_date, 'yyyy-MM-dd'), 365) as first_anniversary
    FROM employees
    ORDER BY hire_date
""")
result.show()

# 按年份分組
print("\n=== 按入職年份分組 ===")
result = spark.sql("""
    SELECT 
        YEAR(TO_DATE(hire_date, 'yyyy-MM-dd')) as hire_year,
        COUNT(*) as employee_count,
        AVG(salary) as avg_salary
    FROM employees
    GROUP BY YEAR(TO_DATE(hire_date, 'yyyy-MM-dd'))
    ORDER BY hire_year
""")
result.show()

## 12. 數據透視和解透視

In [ ]:
# 數據透視
print("=== 數據透視 ===")
result = spark.sql("""
    SELECT *
    FROM (
        SELECT 
            YEAR(TO_DATE(hire_date, 'yyyy-MM-dd')) as hire_year,
            department,
            salary
        FROM employees
    ) 
    PIVOT (
        AVG(salary)
        FOR department IN ('Engineering', 'Sales', 'Marketing')
    )
    ORDER BY hire_year
""")
result.show()

# 使用 DataFrame API 進行透視
print("\n=== 使用 DataFrame API 透視 ===")
pivot_df = spark.sql("""
    SELECT 
        YEAR(TO_DATE(hire_date, 'yyyy-MM-dd')) as hire_year,
        department,
        salary
    FROM employees
""").groupBy("hire_year").pivot("department").avg("salary")

pivot_df.show()

## 13. 複雜分析查詢

In [ ]:
# 移動平均
print("=== 移動平均 ===")
result = spark.sql("""
    SELECT 
        name,
        department,
        salary,
        AVG(salary) OVER (
            PARTITION BY department 
            ORDER BY salary 
            ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING
        ) as moving_avg_salary
    FROM employees
    ORDER BY department, salary
""")
result.show()

# 百分位數
print("\n=== 百分位數分析 ===")
result = spark.sql("""
    SELECT 
        name,
        department,
        salary,
        PERCENT_RANK() OVER (PARTITION BY department ORDER BY salary) as percent_rank,
        NTILE(4) OVER (PARTITION BY department ORDER BY salary) as quartile
    FROM employees
    ORDER BY department, salary
""")
result.show()

In [ ]:
# 部門間比較分析
print("=== 部門間比較分析 ===")
result = spark.sql("""
    WITH dept_stats AS (
        SELECT 
            department,
            COUNT(*) as emp_count,
            AVG(salary) as avg_salary,
            STDDEV(salary) as salary_stddev,
            MIN(salary) as min_salary,
            MAX(salary) as max_salary
        FROM employees
        GROUP BY department
    ),
    overall_stats AS (
        SELECT 
            AVG(avg_salary) as overall_avg,
            STDDEV(avg_salary) as dept_avg_stddev
        FROM dept_stats
    )
    SELECT 
        d.department,
        d.emp_count,
        ROUND(d.avg_salary, 2) as avg_salary,
        ROUND(d.salary_stddev, 2) as salary_stddev,
        ROUND(d.avg_salary - o.overall_avg, 2) as vs_overall_avg,
        CASE 
            WHEN d.avg_salary > o.overall_avg + o.dept_avg_stddev THEN 'High'
            WHEN d.avg_salary < o.overall_avg - o.dept_avg_stddev THEN 'Low'
            ELSE 'Average'
        END as salary_tier
    FROM dept_stats d
    CROSS JOIN overall_stats o
    ORDER BY d.avg_salary DESC
""")
result.show()

## 14. 性能優化技巧

In [ ]:
# 查看執行計劃
print("=== 查看執行計劃 ===")
complex_query = """
    SELECT 
        e.name,
        e.department,
        e.salary,
        d.division,
        AVG(e.salary) OVER (PARTITION BY e.department) as dept_avg_salary
    FROM employees e
    INNER JOIN departments d ON e.department = d.name
    WHERE e.salary > 65000
    ORDER BY e.department, e.salary DESC
"""

result_df = spark.sql(complex_query)
result_df.explain(True)
result_df.show()

In [ ]:
# 使用 CACHE 優化重複查詢
print("=== 使用 CACHE 優化 ===")
spark.sql("CACHE TABLE employees")
spark.sql("CACHE TABLE departments")

# 重複執行相同的查詢會更快
result1 = spark.sql("SELECT COUNT(*) FROM employees")
result2 = spark.sql("SELECT COUNT(*) FROM employees")

print(f"Employee count: {result1.collect()[0][0]}")

# 清除緩存
spark.sql("UNCACHE TABLE employees")
spark.sql("UNCACHE TABLE departments")

## 15. 實戰練習

### 練習1：薪資分析儀表板

In [ ]:
# 創建薪資分析儀表板
print("=== 薪資分析儀表板 ===")

# 1. 總體統計
overall_stats = spark.sql("""
    SELECT 
        COUNT(*) as total_employees,
        ROUND(AVG(salary), 2) as avg_salary,
        ROUND(STDDEV(salary), 2) as salary_stddev,
        MIN(salary) as min_salary,
        MAX(salary) as max_salary,
        COUNT(DISTINCT department) as dept_count
    FROM employees
""")
print("總體統計:")
overall_stats.show()

# 2. 部門薪資分佈
dept_distribution = spark.sql("""
    SELECT 
        department,
        COUNT(*) as emp_count,
        ROUND(AVG(salary), 2) as avg_salary,
        ROUND(MIN(salary), 2) as min_salary,
        ROUND(MAX(salary), 2) as max_salary,
        ROUND(STDDEV(salary), 2) as salary_stddev
    FROM employees
    GROUP BY department
    ORDER BY avg_salary DESC
""")
print("\n部門薪資分佈:")
dept_distribution.show()

# 3. 薪資等級分佈
salary_tiers = spark.sql("""
    SELECT 
        CASE 
            WHEN salary >= 80000 THEN '高薪 (80K+)'
            WHEN salary >= 65000 THEN '中薪 (65K-80K)'
            ELSE '低薪 (<65K)'
        END as salary_tier,
        COUNT(*) as emp_count,
        ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (), 2) as percentage
    FROM employees
    GROUP BY 
        CASE 
            WHEN salary >= 80000 THEN '高薪 (80K+)'
            WHEN salary >= 65000 THEN '中薪 (65K-80K)'
            ELSE '低薪 (<65K)'
        END
    ORDER BY emp_count DESC
""")
print("\n薪資等級分佈:")
salary_tiers.show()

### 練習2：員工流失風險分析

In [ ]:
# 員工流失風險分析
print("=== 員工流失風險分析 ===")

risk_analysis = spark.sql("""
    WITH emp_analysis AS (
        SELECT 
            e.name,
            e.department,
            e.age,
            e.salary,
            DATEDIFF(CURRENT_DATE(), TO_DATE(e.hire_date, 'yyyy-MM-dd')) as days_employed,
            AVG(e.salary) OVER (PARTITION BY e.department) as dept_avg_salary,
            PERCENT_RANK() OVER (PARTITION BY e.department ORDER BY e.salary) as salary_percentile
        FROM employees e
    )
    SELECT 
        name,
        department,
        age,
        salary,
        ROUND(days_employed / 365.0, 1) as years_employed,
        ROUND(dept_avg_salary, 2) as dept_avg_salary,
        ROUND(salary_percentile * 100, 1) as salary_percentile,
        CASE 
            WHEN salary_percentile < 0.25 AND days_employed > 365 THEN 'High Risk'
            WHEN salary_percentile < 0.5 AND days_employed > 730 THEN 'Medium Risk'
            WHEN age > 30 AND salary_percentile < 0.75 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as flight_risk
    FROM emp_analysis
    ORDER BY 
        CASE flight_risk
            WHEN 'High Risk' THEN 1
            WHEN 'Medium Risk' THEN 2
            ELSE 3
        END,
        salary_percentile
""")
risk_analysis.show()

## 16. DataFrame 與 SQL 的對比

In [ ]:
# 比較 DataFrame API 和 SQL 的等效操作
print("=== DataFrame API vs SQL 比較 ===")

# SQL 方式
sql_result = spark.sql("""
    SELECT department, AVG(salary) as avg_salary
    FROM employees
    WHERE salary > 65000
    GROUP BY department
    ORDER BY avg_salary DESC
""")

print("SQL 結果:")
sql_result.show()

# DataFrame API 方式
df_result = employees_df.filter(col("salary") > 65000) \
                       .groupBy("department") \
                       .agg(avg("salary").alias("avg_salary")) \
                       .orderBy(col("avg_salary").desc())

print("DataFrame API 結果:")
df_result.show()

# 驗證結果相同
print(f"結果相同: {sql_result.collect() == df_result.collect()}")

## 17. 總結和清理

In [ ]:
# 顯示學習總結
print("=== Spark SQL 學習總結 ===")
print("✓ 基本 SQL 查詢：SELECT, WHERE, GROUP BY, ORDER BY")
print("✓ 高級查詢：子查詢, CTE, 窗口函數")
print("✓ 連接操作：INNER JOIN, LEFT JOIN, 多表連接")
print("✓ 字符串和日期函數")
print("✓ 條件表達式：CASE WHEN")
print("✓ 數據透視和解透視")
print("✓ 性能優化：CACHE, EXPLAIN")
print("✓ 實戰分析：薪資分析, 風險評估")
print("✓ DataFrame API 與 SQL 對比")

# 清理臨時視圖
spark.sql("DROP VIEW IF EXISTS employees")
spark.sql("DROP VIEW IF EXISTS departments")
spark.sql("DROP VIEW IF EXISTS projects")

# 停止 SparkSession
spark.stop()
print("\nSpark session 已停止")

## 學習重點回顧

### 核心概念
1. **臨時視圖**：createOrReplaceTempView() 創建可查詢的表
2. **SQL 語法**：完整的 SQL 支持，包括複雜查詢
3. **執行計劃**：explain() 方法優化查詢性能
4. **緩存策略**：CACHE TABLE 提升重複查詢性能

### 高級特性
1. **窗口函數**：ROW_NUMBER, RANK, LAG, LEAD
2. **CTE**：WITH 子句簡化複雜查詢
3. **數據透視**：PIVOT 進行數據重組
4. **條件邏輯**：CASE WHEN 實現複雜業務邏輯

### 實戰應用
1. **數據分析**：統計分析、分組聚合
2. **報表生成**：儀表板、KPI 指標
3. **業務邏輯**：風險評估、分類分析
4. **性能優化**：查詢調優、執行計劃分析

掌握 Spark SQL 後，您可以用熟悉的 SQL 語法處理大規模數據，並享受 Spark 的分散式計算能力。